In [1]:
!pip install langchain
!pip install openai
!pip install -U huggingface-hub
!pip install streamlit

  Using cached huggingface_hub-1.1.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.1.2-py3-none-any.whl (514 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.0.1 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.1.2 which is incompatible.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.2 which is incompatible.


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf"

In [3]:
from langchain import PromptTemplate
template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘?'

In [4]:
from langchain_openai import ChatOpenAI

llm1 = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",  
)

prompt = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
print(llm1.invoke(prompt))

content='진희가 키우고 있는 동물은 강아지입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 29, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None} id='run-153b75ef-46f9-4965-8ee4-4b6db7684417-0' usage_metadata={'input_tokens': 29, 'output_tokens': 15, 'total_tokens': 44}


In [5]:
from langchain_huggingface import HuggingFaceEndpoint

llm2 = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",    # xxl 대신 large 권장 (무료 서버에서 안정적)
    task="text2text-generation",       # task 필수
    temperature=0.8,
    max_new_tokens=512
)

prompt = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
print(llm2.invoke(prompt))

C:\Users\chogu\anaconda3\envs\11m\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'InferenceClient' object has no attribute 'post'

In [6]:
from langchain.model_laboratory import ModelLaboratory
model_lab = ModelLaboratory.from_llms([llm1, llm2])
model_lab.compare("대한민국의 가을은 몇 월부터 몇 월까지야?")

Input:
대한민국의 가을은 몇 월부터 몇 월까지야?

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F920883580> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F9208989D0> root_client=<openai.OpenAI object at 0x000001F91FF9ED60> root_async_client=<openai.AsyncOpenAI object at 0x000001F920883430> model_name='gpt-4o-mini' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy=''
대한민국의 가을은 일반적으로 9월부터 11월까지로 여겨집니다. 9월 중순부터 11월 말까지가 가을의 주요 시기로, 이 시기에 기온이 서늘해지고 단풍이 아름답게 물드는 특징이 있습니다.

HuggingFaceEndpoint
Params: {'endpoint_url': None, 'task': 'text2text-generation', 'model_kwargs': {}}


AttributeError: 'InferenceClient' object has no attribute 'post'

In [7]:
!pip install pypdf
!pip install tiktoken
!pip install faiss-cpu
!pip install sentence-transformers

  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.2
    Uninstalling huggingface_hub-1.1.2:
      Successfully uninstalled huggingface_hub-1.1.2


In [8]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/chogu/OneDrive/바탕 화면/대학교/4학년/모바일 프로그래밍 실습/10주차/2/080413-main/Data/The_Adventures_of_Tom_Sawyer.pdf")
document = loader.load()

document[5].page_content[:500]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he \ndidn’t like working. He liked playing and having \nadventures. One Friday, he didn’t go to school—he went \nto the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you \ndidn’t go to school today. Tomorrow you’re going to work \nfor me. You can paint the fence.” \nSaturday morning, Tom was not happy, '

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk"
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()  # 임베딩 처리
db = FAISS.from_documents(document, embeddings)

In [12]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "sk"

llm = ChatOpenAI(
    temperature=0,       # 창의성 0일 때 일관된 출력
    model_name="gpt-4o-mini",  # 모델명
)

prompt = PromptTemplate(
    input_variables=["country"],
    template="{country}의 수도는 어디야?",
)

chain = LLMChain(llm=llm, prompt=prompt)  # 체인으로 llm과 프롬프트 연결
chain.run("대한민국") 

'대한민국의 수도는 서울입니다.'

In [14]:
prompt1 = PromptTemplate(
    input_variables=['sentence'],
    template="다음 문장을 한글로 번역하세요.\n\n{sentence}"
)

# 번역에 대한 LLMChain
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="translation")

# 프롬프트2 정의
prompt2 = PromptTemplate.from_template(
    "다음 문장을 한 문장으로 요약하세요.\n\n{translation}"
)

# 요약에 대한 LLMChain
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="summary")

from langchain.chains import SequentialChain
all_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=['sentence'],
    output_variables=['translation', 'summary']
)

sentence = """
One limitation of LLMs is their lack of contextual information (e.g.,
access to some specific documents or emails). You can combat this by giving
LLMs access to the specific external data.
In fact, this, you first need to load the external data with a document loader.
Langchain provides a variety of loaders for different types of documents
ranging from PDFs and emails to websites and YouTube videos.
"""

all_chain.invoke({"sentence": sentence})

{'sentence': '\nOne limitation of LLMs is their lack of contextual information (e.g.,\naccess to some specific documents or emails). You can combat this by giving\nLLMs access to the specific external data.\nIn fact, this, you first need to load the external data with a document loader.\nLangchain provides a variety of loaders for different types of documents\nranging from PDFs and emails to websites and YouTube videos.\n',
 'translation': 'LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보가 부족하다는 점입니다. 이를 해결하기 위해 LLM에 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다. 사실, 이를 위해서는 먼저 문서 로더를 사용하여 외부 데이터를 로드해야 합니다. Langchain은 PDF, 이메일, 웹사이트, YouTube 비디오 등 다양한 유형의 문서에 대한 다양한 로더를 제공합니다.',
 'summary': 'LLM의 맥락 정보 부족 문제를 해결하기 위해 Langchain은 PDF, 이메일, 웹사이트, YouTube 비디오 등 다양한 외부 데이터를 로드할 수 있는 문서 로더를 제공합니다.'}

In [17]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    temperature=0,        # 창의성 0으로 설정
    model_name="gpt-4o-mini",   # 모델명
)

from langchain import ConversationChain
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="진희는 강아지를 한 마리 키우고 있습니다.")
conversation.predict(input="영수는 고양이 두 마리를 키우고 있습니다.")
conversation.predict(input="진희와 영수가 키우는 동물은 총 몇마리?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 진희는 강아지를 한 마리 키우고 있습니다.
AI:


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 진희는 강아지를 한 마리 키우고 있습니다.
AI: 아, 진희가 강아지를 키우고 있다니 정말 귀엽네요! 어떤 종류의 강아지를 키우고 있는지 궁금해요. 강아지의 이름은 무엇인가요? 그리고 진희는 강아지와 함께 어떤 활동을 즐기나요? 산책이나 놀이, 훈련 같은 것들이요!
Human: 영수는 고양이 두 마리를 키우고 있습니다.
AI:


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 진희는 강아지를 한 마리 키우고 있습니다.
AI: 아, 진희가 강아지를 키우고 있다니 정말 귀엽네요! 어떤 종류의 강아지를 키우고 있는지 궁금해요. 강아지의 이름은 무엇인가요? 그리고 진희는 강아지와 함께 어떤 활동을 즐기나요? 산책이나 놀이, 훈련 같은 것들이요!
Human: 영수는 고양이 두 마리를 키우고 있습니다.
AI: 오, 영수가 고양이 두 마리를 키우고 있다니 정말 멋지네요! 고양이들은 독립적이면서도 사랑스러운 친구들이죠. 어떤 종류의 고양이인지 궁금해요. 이름은 무엇인가요? 그리고 영수는 고양이들과 어떤 활동을 즐기나요? 예를 들어, 장난감으로 놀아주거나, 고양이와 함께 편안한 시간을 보내는 것 같은 것들이요!
Human: 진희와 영수가 키우는 동물은 총 몇마리?
AI:

> Finished chain.


'진희는 강아지 한 마리를 키우고 있고, 영수는 고양이 두 마리를 키우고 있으니, 총 동물의 수는 1 + 2 = 3마리입니다! 진희와 영수가 각각의 반려동물과 함께하는 모습이 상상이 가네요. 그들은 서로 다른 매력을 가진 동물들과 함께 즐거운 시간을 보내고 있을 것 같아요!'

In [20]:
!pip install wikipedia
!pip install numexpr

In [24]:
import sys
print(sys.executable)

C:\Users\chogu\anaconda3\envs\11m\python.exe


In [26]:
!"C:\Users\chogu\anaconda3\envs\11m\python.exe" -m pip install wikipedia numexpr

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11705 sha256=0eaaeaf4c94fa3f1d9121f1ddb844294c3436e0d32dc0320bb7688609556d24d
  Stored in directory: c:\users\chogu\appdata\local\pip\cache\wheels\07\93\05\72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [27]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    temperature=0,       # 창의성 0으로 설정
    model_name="gpt-4o-mini",  # 모델명
)

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)  
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    description="계산이 필요할 때 사용",
    verbose=True
)

agent.run("에드 시런이 태어난 해는? 2024년도 현재 에드시런은 몇 살?")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15692\395289981.py:12: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


에드 시런의 출생 연도를 알아야 그의 나이를 계산할 수 있습니다. 먼저 에드 시런의 출생 연도를 확인해보겠습니다.  
Action: wikipedia  
Action Input: "Ed Sheeran"  
Observation: Page: Ed Sheeran
Summary: Edward Christopher Sheeran ( SHEER-ən; born 17 February 1991) is an English singer-songwriter. Born in Halifax, West Yorkshire, and raised in Framlingham, Suffolk, he began writing songs around the age of eleven. In early 2011, Sheeran independently released the extended play No. 5 Collaborations Project. He signed with Asylum Records the same year.
Sheeran's debut album, + ("Plus"), was released in September 2011 and topped the UK Albums Chart. It contained his first hit single, "The A Team". In 2012, Sheeran won the Brit Awards for Best British Male Solo Artist and British Breakthrough Act. Sheeran's second studio album, × ("Multiply"), topped charts around the world upon its release in June 2014. It was named the second-best-selling album worldwide of 2015. In the same year, × won Album of the Year at the 2015 Brit Awards, and he r

'에드 시런은 1991년에 태어났으며, 2024년 현재 33세입니다.'